## Data Preparation
Let's load this data into Python.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', encoding='latin-1', usecols=['user_id', 'movie_id', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', encoding='latin-1', usecols=['movie_id', 'title', 'genres'])

In [2]:
print(movies.shape)

(100, 3)


## Content-Based Recommendation Model

Computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

In [3]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
print(movies.shape)


(100, 3)


In [4]:
# Convert genres to string value
def clean(text):
    return text.replace("\'[", '').replace("]\'", '').replace("\\", '').replace("\"[", '').replace("]\"", '')
movies['genres'] = movies['genres'].fillna("").astype('str')
movies['genres'] = [clean(x) for x in movies['genres']]
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),"['Animation', ""Children's"", 'Comedy']"
1,2,Jumanji (1995),"['Adventure', ""Children's"", 'Fantasy']"
2,3,Grumpier Old Men (1995),"['Comedy', 'Romance']"
3,4,Waiting to Exhale (1995),"['Comedy', 'Drama']"
4,5,Father of the Bride Part II (1995),['Comedy']


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape

(100, 44)

In [6]:
tfidf_matrix

<100x44 sparse matrix of type '<class 'numpy.float64'>'
	with 231 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
#cosine_sim[:4, :4]
cosine_sim

array([[1.        , 0.15337409, 0.12551391, ..., 0.        , 0.        ,
        0.        ],
       [0.15337409, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12551391, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.25861841],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.25861841, 0.        ,
        1.        ]])

In [8]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

#TODO: Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    index = indices[title]
    cos_score = cosine_sim[index]
    result = pd.DataFrame(data=[titles, cos_score], index=['Title', 'Similarity score']).transpose()
    return result.sort_values(by='Similarity score', ascending=False)[1:]

Let's try and get the top recommendations for a few movies and see how good the recommendations are.

In [9]:
genre_recommendations('Toy Story (1995)').head(20)

,Title,Similarity score
12,Balto (1995),0.819159
86,Dunston Checks In (1996),0.702985
33,Babe (1995),0.582071
47,Pocahontas (1995),0.50019
68,Friday (1995),0.275812
87,Black Sheep (1996),0.275812
37,It Takes Two (1995),0.275812
51,Mighty Aphrodite (1995),0.275812
64,Bio-Dome (1996),0.275812
18,Ace Ventura: When Nature Calls (1995),0.275812


## Collaborative Filtering Recommendation Model


Use the file **ratings.csv** first as it contains User ID, Movie IDs and Ratings. These three elements are all needed for determining the similarity of the users based on their ratings for a particular movie.


In [10]:
# Fill NaN values in user_id and movie_id column with 0
ratings['user_id'] = ratings['user_id'].fillna(0)
ratings['movie_id'] = ratings['movie_id'].fillna(0)

# Replace NaN values in rating column with average of all values
ratings['rating'] = ratings['rating'].fillna(ratings['rating'].mean())

Take a random sample of 20,000 ratings (2%) (due to limitation of personal laptop)

In [11]:
# Randomly sample 1% of the ratings dataset
small_data = ratings.sample(frac=0.02)
# Check the sample info
print(small_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 7299 to 3783
Data columns (total 3 columns):
user_id     150 non-null int64
movie_id    150 non-null int64
rating      150 non-null int64
dtypes: int64(3)
memory usage: 4.7 KB
None


In [12]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(small_data, test_size=0.2)

In [13]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = train_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])
test_data_matrix = test_data.as_matrix(columns = ['user_id', 'movie_id', 'rating'])

# Check their shape
print(train_data_matrix.shape)
print(test_data_matrix[:4, :4])

(120, 3)
[[93  9  3]
 [26  1  4]
 [15 23  3]
 [ 5 68  4]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Now I use the **pairwise_distances** function from sklearn [Pearson Correlation Coefficient](https://stackoverflow.com/questions/1838806/euclidean-distance-vs-pearson-correlation-vs-cosine-similarity). This method provides a safe way to take a distance matrix as input, while preserving compatibility with many other algorithms that take a vector array.

In [14]:
from sklearn.metrics.pairwise import pairwise_distances

# User Similarity Matrix
user_correlation = 1 - pairwise_distances(train_data, metric='correlation')
user_correlation[np.isnan(user_correlation)] = 0
print(user_correlation[:4, :4])

[[1.         0.99741697 0.16275769 0.98876226]
 [0.99741697 1.         0.09146616 0.99694642]
 [0.16275769 0.09146616 1.         0.01342554]
 [0.98876226 0.99694642 0.01342554 1.        ]]


In [15]:
# Item Similarity Matrix
item_correlation = 1 - pairwise_distances(train_data_matrix.T, metric='correlation')
item_correlation[np.isnan(item_correlation)] = 0
print(item_correlation[:4, :4])

[[ 1.         -0.06774789  0.00958159]
 [-0.06774789  1.          0.01650567]
 [ 0.00958159  0.01650567  1.        ]]


With the similarity matrix in hand, I can now predict the ratings that were not included with the data. Using these predictions, I can then compare them with the test data to attempt to validate the quality of our recommender model.

For the user-user CF case, I will look at the similarity between 2 users (A and B, for example) as weights that are multiplied by the ratings of a similar user B (corrected for the average rating of that user).

In [16]:
#TODO: Function to predict ratings
def predict(ratings, similarity, type='user'):
    pred = np.zeros(ratings.shape)
    if type == 'user':
        for row, rating in enumerate(ratings):
            pred[row] = [rating[0], rating[1], np.sum(np.dot(similarity[row][row:], ratings[:,2][row:])) / np.sum(similarity[row][row:])]
    elif type == 'item':
        for col, rating in enumerate(ratings.T):
            if col == 2:
              pred[:, col] = np.sum(np.dot(np.array([similarity[:, col]]).T, np.array([ratings[:, col]])), axis=0)
            else:
              pred[:, col] = ratings[:, col]
    return pred

In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def rmse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    print(pred)
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

In [18]:
# Predict ratings on the training data with both similarity score
user_prediction = predict(train_data_matrix, user_correlation, type='user')
item_prediction = predict(train_data_matrix, item_correlation, type='item')

# RMSE on the test data
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

[87.         57.          3.42437795 93.         56.          3.42786999
  4.         74.          3.45647506 90.         48.          3.40625329
 54.         41.          3.40564126 46.         28.          3.38198021
 26.         70.          3.42541598 34.          6.          3.37534569
  1.         52.          3.45805857 34.         50.          3.43982711
 73.         12.          3.41191637 72.         34.          3.46344441
 74.         84.          3.47838789  7.         74.          3.44972025
 66.         60.          3.45430761 98.         61.          3.49364612
 11.         40.          3.44731604 10.         72.          3.41312009
 66.         91.          3.44743335 51.         19.          3.49042237
 71.         76.          3.42575075 98.          4.          3.49374223
  9.         63.          3.38806515 15.         27.          3.39293789
 60.         44.          3.40799639 49.         93.          3.39979371
 68.         93.          3.42678222 34.         16